# Pandas Challenge - PyCity Schools

Martin Singla - Data Analyst
Data Analytics Bootcamp - Tecnológico de Monterrey / Trinity Education

In the following Jupyter notebook we utilize Python's Pandas dependendency to analyze education trends at a school district level. 

In [24]:
#Load dependencies and data
import pandas as pd

schools = pd.read_csv("Resources/schools_complete.csv", encoding = "utf-8")
students = pd.read_csv("Resources/students_complete.csv", encoding = "utf-8")

In [25]:
#Checking data frames: Schools
schools.head() #14 schools with its name, type, size and budget

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [26]:
#Checking data frames: Students
students.head() #39,170 students w/ its name, gender, grade, school, reading and math scores.

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [52]:
#High level district statistics summary 
df1 = pd.DataFrame({
    "Total Schools": [schools["school_name"].count()] ,
    "Total Students": [students["student_name"].count()], 
    "Total Budget": [f'${schools["budget"].sum()}'], 
    "Avg.Math Score": [round(students["math_score"].mean(), 2)], 
    "Avg.Reading Score": [round(students["reading_score"].mean(), 2)], 
    "% Passing Math": [f'% {round(students.loc[students["math_score"] >= 70,"math_score"].count() / students["math_score"].count() * 100, 2)}'], 
    "% Passing Reading": [f'% {round(students.loc[students["reading_score"] >= 70,"reading_score"].count() / students["math_score"].count() * 100, 2)}'], 
    "% Overall Passing": [f'% {round(students.loc[(students["reading_score"] >= 70) & (students["math_score"] >= 70), "Student ID"].count() / students["Student ID"].count()*100,2)}']
})

df1

,Total Schools,Total Students,Total Budget,Avg.Math Score,Avg.Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,$24649428,78.99,81.88,% 74.98,% 85.81,% 65.17


Top level summary statistics indicate that students are perfroming better at reading classes than math classes. I am personally very familiar with this trend!